## Sandbox

Un  `.ipynb` pour tester des idées :)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));

  Activating project at `c:\cours_MASTER\PROJ\workspace\PROJ`


In [2]:
infos = open(
    readInstance,
    joinpath("data", "20_USA-road-d.BAY.gr"),
);
graph = graphFromData(infos);

In [3]:
# foreach(((key, value),) -> println("$key = $value"), pairs(infos))

In [4]:
# foreach(i -> println(graph[i]), labels(graph))

In [5]:
# foreach(((i, j),) -> println(graph[i, j]), edge_labels(graph))

In [6]:
function flow_creation(i::Int64)::Int64
    return Int64(graph[i].is_s) - Int64(graph[i].is_t)
end

flow_creation (generic function with 1 method)

In [7]:
# __________MODELE_MAITRE__________
#declaration du modele
modele_maitre = Model(Gurobi.Optimizer)
modele_maitre = Model(
                optimizer_with_attributes(
                Gurobi.Optimizer, "Presolve" => 0
                ))
set_silent(modele_maitre)
#declaration des variables
@variable(modele_maitre, z >= 0)
@variable(modele_maitre, a[edge_labels(graph)], Bin)  # vaut 1 ssi arc (i,j) choisi
@objective(modele_maitre, Min, z)
# declaration des contraintes
@constraint(modele_maitre, sum(graph[i, j].d * a[(i, j)] for (i, j) in edge_labels(graph)) <= z )
# Conservation du flot :
@constraint(
modele_maitre,
flow_cons[i in labels(graph)],
(
    sum(a[(i, j)] for j in outneighbor_labels(graph, i))
    -
    sum(a[(j, i)] for j in inneighbor_labels(graph, i))
    == flow_creation(i)
))
@constraint(modele_maitre, sum(graph[i].p * a[(i, j)] for (i, j) in edge_labels(graph))
+ graph[graph[].t].p <= graph[].big_s )
#__________________________________

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Presolve to value 0


14 a[(1, 2)] + 14 a[(1, 3)] + 14 a[(1, 4)] + 14 a[(1, 5)] + 14 a[(1, 6)] + 14 a[(1, 7)] + 14 a[(1, 8)] + 14 a[(1, 11)] + 14 a[(1, 12)] + 14 a[(1, 14)] + 14 a[(1, 18)] + 14 a[(1, 19)] + 14 a[(1, 20)] + 4 a[(2, 1)] + 4 a[(2, 9)] + 4 a[(2, 13)] + 7 a[(3, 1)] + 7 a[(3, 4)] + 7 a[(3, 9)] + 7 a[(3, 11)] + 7 a[(3, 13)] + 7 a[(3, 19)] + 9 a[(4, 1)] + 9 a[(4, 3)] + 9 a[(4, 9)] + 9 a[(4, 10)] + 9 a[(4, 13)] + 9 a[(4, 14)] + 9 a[(4, 16)] + 9 a[(5, 1)] + [[...76 terms omitted...]] + 7 a[(15, 11)] + 7 a[(15, 19)] + 7 a[(16, 4)] + 7 a[(16, 5)] + 7 a[(16, 11)] + 7 a[(16, 12)] + 7 a[(16, 19)] + 7 a[(16, 20)] + 8 a[(18, 1)] + 8 a[(18, 5)] + 8 a[(18, 7)] + 8 a[(18, 9)] + 8 a[(18, 12)] + 8 a[(18, 13)] + 8 a[(18, 20)] + 12 a[(19, 1)] + 12 a[(19, 3)] + 12 a[(19, 9)] + 12 a[(19, 10)] + 12 a[(19, 13)] + 12 a[(19, 14)] + 12 a[(19, 16)] + 9 a[(20, 1)] + 9 a[(20, 6)] + 9 a[(20, 9)] + 9 a[(20, 10)] + 9 a[(20, 13)] + 9 a[(20, 16)] + 9 a[(20, 17)] + 9 a[(20, 18)] <= 59

In [8]:
#__________SP1__________
#declaration du modele
SP1 = Model(Gurobi.Optimizer)
SP1 = Model(
                optimizer_with_attributes(
                Gurobi.Optimizer, "Presolve" => 0
                ))
set_silent(SP1)
#declaration des variables
@variable(SP1, delta_d[edge_labels(graph)] >= 0)
#declaration des contraintes
@constraint(SP1, [(i, j) in edge_labels(graph)], delta_d[(i,j)] <= graph[i,j].big_d )
@constraint(SP1, sum(delta_d[(i,j)] for (i, j) in edge_labels(graph)) <= graph[].d1 )
#_______________________

#__________SP2__________
#declaration du modele
SP2 = Model(Gurobi.Optimizer)
SP2 = Model(
                optimizer_with_attributes(
                Gurobi.Optimizer, "Presolve" => 0
                ))
set_silent(SP2)
#declaration des variables
@variable(SP2, delta_p[labels(graph)] >= 0)
#declaration des contraintes
@constraint(SP2, [i in labels(graph)], delta_p[i] <= 2 )
@constraint(SP2, sum(delta_p[i] for i in labels(graph)) <= graph[].d2 )
#_______________________

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Presolve to value 0
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Presolve to value 0


delta_p[1] + delta_p[2] + delta_p[3] + delta_p[4] + delta_p[5] + delta_p[6] + delta_p[7] + delta_p[8] + delta_p[9] + delta_p[10] + delta_p[11] + delta_p[12] + delta_p[13] + delta_p[14] + delta_p[15] + delta_p[16] + delta_p[17] + delta_p[18] + delta_p[19] + delta_p[20] <= 5

In [9]:
function heuristic_SP1(a_val)
    SP1_value = 0
    worst_weight::Dict{Tuple{Int64,Int64}, Float64} = Dict{Tuple{Int64,Int64},Float64}()
    delta_d_heur::Dict{Tuple{Int64,Int64},Float64} = Dict{Tuple{Int64,Int64},Float64}()
    for (i, j) in edge_labels(graph)
        if (a_val[(i, j)] > 0)
            worst_weight[(i,j)] = graph[i, j].d * graph[i, j].big_d
            SP1_value += graph[i, j].d
        end
        delta_d_heur[(i , j)] = 0
    end
    tuple_vec = sort(collect(worst_weight), by=x -> x.second, rev=true)
    
    d1::Float64 = graph[].d1
    rem::Float64 = 0
    for vec in tuple_vec
        i, j = vec.first
        rem = min(d1, graph[i, j].big_d)
        delta_d_heur[(i, j)] = rem
        SP1_value += graph[i, j].d * rem
        d1 -= rem
        if (d1 == 0)
            break
        end
    end
    return SP1_value, delta_d_heur
end

function heuristic_SP2(a_val)
    SP2_value = 0
    worst_weight::Dict{Int64, Float64} = Dict{Int64,Float64}()
    delta_p_heur::Dict{Int64,Float64} = Dict{Int64,Float64}()
    for (i, j) in edge_labels(graph)
        if (a_val[(i, j)] > 0)
            worst_weight[i] = 2 * graph[i].ph
            SP2_value += graph[i].p
        end
        delta_p_heur[i] = 0
    end
    t::Float64 = graph[].t
    worst_weight[t] = 2 * graph[t].ph
    SP2_value += graph[t].p
    delta_p_heur[t] = 0
    tuple_vec = sort(collect(worst_weight), by=x -> x.second, rev=true)
    
    d2::Float64 = graph[].d2
    rem::Float64 = 0
    for vec in tuple_vec
        i = vec.first
        rem = min(d2, 2)
        delta_p_heur[i] = rem
        SP2_value += graph[i].ph * rem
        d2 -= rem
        if (d2 == 0)
            break
        end
    end
    return SP2_value, delta_p_heur
end

function plans_coupants(
    graph::MetaGraph;
    timelimit::Float64=3600.0
)::ModResultWrapper
    start = time()
    s::Int64 = graph[].s
    t::Int64 = graph[].t
    cpt = 0
    best_sup::Float64 = Inf
    while (time() - start < timelimit )
        JuMP.optimize!(modele_maitre)
        m_value = JuMP.objective_value(modele_maitre)
        Z = JuMP.value(z)
        A = JuMP.value.(a)
        for (i, j) in edge_labels(graph)
            if A[(i,j)] > 0
                println("(",i,",",j,") = ", A[(i,j)])
            end
        end
        # println("distance : ", sum(graph[i, j].d * A[(i, j)] for (i, j) in edge_labels(graph)))
        # println("weights : ", sum(graph[i].p * A[(i, j)] for (i, j) in edge_labels(graph)) + graph[graph[].t].p)
        # println("Z: ", Z)
        println("Objective value modele_maitre: ", m_value)
        cpt += 1
        # # Résolution SP1
        # @objective(SP1, Max, sum(graph[i, j].d * (1 + delta_d[(i,j)]) * A[(i, j)] for (i, j) in edge_labels(graph)) )
        # JuMP.optimize!(SP1)
        # SP1_value = JuMP.objective_value(SP1)
        # println("Objective value SP1: ", SP1_value)
        # delta_d_sol = JuMP.value.(delta_d)
        SP1_value, delta_d_sol = heuristic_SP1(A)
        println("Objective value SP1: ", SP1_value)
        if (round(SP1_value - Z; digits=4) > 0)
            @constraint(modele_maitre,
            sum(graph[i, j].d * (1 + delta_d_sol[(i,j)]) * a[(i, j)] for (i, j) in edge_labels(graph)) <= z )
            if (SP1_value < best_sup)
                best_sup = SP1_value
            end
            @constraint(modele_maitre, z <= best_sup )
            println("Violée 1")
        end
        # # Résolution SP2
        # @objective(SP2, Max,
        # sum((graph[i].p + delta_p[i] * graph[i].ph) * A[(i, j)] for (i, j) in edge_labels(graph)) +
        # graph[t].p + delta_p[t] * graph[t].ph )
        # JuMP.optimize!(SP2)
        # # temp = JuMP.value.(delta_d)
        # # for (i, j) in edge_labels(graph)
        # #     if temp[(i,j)] > 0
        # #         println("(",i,",",j,") = ", temp[(i,j)])
        # #     end
        # # end
        # SP2_value = JuMP.objective_value(SP2)
        # println("Objective value SP2: ", SP2_value)
        # delta_p_sol = JuMP.value.(delta_p)
        SP2_value, delta_p_sol = heuristic_SP2(A)
        println("Objective value SP2: ", SP2_value)
        if (SP2_value > graph[].big_s)
            @constraint(modele_maitre,
            sum((graph[i].p + delta_p_sol[i] * graph[i].ph) * a[(i, j)] for (i, j) in edge_labels(graph)) +
            graph[t].p + delta_p_sol[t] * graph[t].ph <= graph[].big_s )
            println("Violée 2")
        end
        if (round(SP1_value - Z; digits=4) <= 0 && SP2_value <= graph[].big_s)
            # println("C'est bon wtf")
            break
        end
    end
    println("COMPTEUR = ", cpt)
    println("elapsed time : ", time() - start)
    return (
        primal_status=primal_status(modele_maitre),
        dual_status=dual_status(modele_maitre),
        term_status=termination_status(modele_maitre),
        obj_value=objective_value(modele_maitre),
        bound=objective_bound(modele_maitre),
        a=Dict{Tuple{Int64, Int64}, Float64}((i, j) => value(a[(i, j)]) for (i, j) in edge_labels(graph)),
    )
end


plans_coupants (generic function with 1 method)

In [10]:
res = plans_coupants(graph; timelimit=1200.0)
println(mkPath(res.a))
println(res.obj_value)

(1,5) = 1.0
(5,17) = 1.0
(15,19) = 1.0
(19,1) = 1.0
Objective value modele_maitre: 9365.0
Objective value SP1: 15332.560000000001
Violée 1
Objective value SP2: 69.0
Violée 2
(1,20) = 1.0
(4,1) = 1.0
(15,4) = 1.0
(20,17) = 1.0
Objective value modele_maitre: 9365.0
Objective value SP1: 15332.560000000001
Violée 1
Objective value SP2: 63.0
(1,12) = 1.0
(11,1) = 1.0
(12,17) = 1.0
(15,11) = 1.0
Objective value modele_maitre: 9365.0
Objective value SP1: 15332.560000000001
Violée 1
Objective value SP2: 69.0
Violée 2
(1,5) = 1.0
(4,1) = 1.0
(5,17) = 1.0
(15,4) = 1.0
Objective value modele_maitre: 12480.419999999984
Objective value SP1: 15332.560000000001
Violée 1
Objective value SP2: 63.0
(9,20) = 1.0
(11,9) = 1.0
(15,11) = 1.0
(20,17) = 1.0
Objective value modele_maitre: 12480.419999999982
Objective value SP1: 15332.560000000001
Violée 1
Objective value SP2: 69.0
Violée 2
(5,17) = 1.0
(11,13) = 1.0
(13,5) = 1.0
(15,11) = 1.0
Objective value modele_maitre: 12480.42
Objective value SP1: 15332.5